In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import numpy as np

In [3]:
mnist=input_data.read_data_sets("../MNIST_data/",one_hot=True)
print(mnist.train.images.shape,mnist.train.labels.shape)

NameError: name 'input_data' is not defined

In [5]:
sess = tf.InteractiveSession()
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

In [2]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [7]:
#第一层卷积
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
#第二层卷积
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
#密集连接
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
#dropout
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
#输出层
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [ ]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
corrcet_result=tf.argmax(y_conv,1)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.global_variables_initializer())
for i in range(1000):
  batch = mnist.train.next_batch(50)
  if i%10 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print(accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.18
step 10, training accuracy 0.38
step 20, training accuracy 0.44
step 30, training accuracy 0.72
step 40, training accuracy 0.68
step 50, training accuracy 0.52
step 60, training accuracy 0.8
step 70, training accuracy 0.84
step 80, training accuracy 0.88
step 90, training accuracy 0.9
step 100, training accuracy 0.86
step 110, training accuracy 0.88
step 120, training accuracy 0.82
step 130, training accuracy 0.78
step 140, training accuracy 0.82
step 150, training accuracy 0.84
step 160, training accuracy 0.84
step 170, training accuracy 0.88
step 180, training accuracy 0.78
step 190, training accuracy 0.92
step 200, training accuracy 0.9
step 210, training accuracy 0.92
step 220, training accuracy 0.94
step 230, training accuracy 0.88
step 240, training accuracy 0.92
step 250, training accuracy 0.88
step 260, training accuracy 0.92
step 270, training accuracy 0.9
step 280, training accuracy 0.88
step 290, training accuracy 0.88
step 300, training accura

In [1]:
test_x,test_y=mnist.train.next_batch(15)
test_x,test_y=mnist.train.next_batch(15)
print(corrcet_result.eval({x:test_x}))
fig = plt.figure()
for i in range(0,15,1):
    image=test_x[i].reshape(28,28)
    lable=np.argmax(test_y[i])
    plt.subplot(3,5,i+1) # 绘制前15个手写体数字，以3行6列子图形式展示
    plt.tight_layout() # 自动适配
    plt.imshow(image, cmap='Greys') # 使用灰色显示像素灰度值
    plt.title("Label: {}".format(lable)) # 设置标签为子图标题

NameError: name 'mnist' is not defined